In [1]:
#!jupyter nbconvert --to script #1.ipynb

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

In [3]:
pd.set_option('display.max_rows', 1000)

In [4]:
bene = pd.read_csv("Train_Beneficiarydata-1542865627584.csv")
inpat = pd.read_csv("Train_Inpatientdata-1542865627584.csv")
outpat = pd.read_csv("Train_Outpatientdata-1542865627584.csv")
train_classification = pd.read_csv("Train-1542865627584.csv")

In [ ]:
#Train - bene
print(bene.head(1))
print(bene.shape)
print(bene.dtypes)
print(bene.isnull().sum())

In [ ]:
#Train - in patients
print(inpat.head(1))
print(inpat.shape)
print(inpat.dtypes)
print(inpat.isnull().sum())

In [ ]:
#Train - Outpatients
print(outpat.head(1))
print(outpat.shape)
print(outpat.dtypes)
print(outpat.isnull().sum())

In [ ]:
# train_classification 
print(train_classification.head(1))
print(train_classification.shape)
print(train_classification.dtypes)
print(train_classification.isnull().sum())

In [9]:
inpat_bene = pd.merge(inpat, bene, left_on = 'BeneID',right_on = 'BeneID', how = 'left')

In [10]:
outpat_bene = pd.merge(outpat, bene, left_on = 'BeneID',right_on = 'BeneID', how = 'left')

In [ ]:
train = pd.concat([inpat_bene, outpat_bene])

In [12]:
train['DOB'] = pd.to_datetime(train['DOB'])
train['DOD'] = pd.to_datetime(train['DOD'])
train['ClaimEndDt'] = pd.to_datetime(train['ClaimEndDt'])
train['ClaimStartDt'] = pd.to_datetime(train['ClaimStartDt'])
train['AdmissionDt'] = pd.to_datetime(train['AdmissionDt'])
train['DischargeDt'] = pd.to_datetime(train['DischargeDt'])

In [ ]:
#Preprocessing for groupby
train['deceased_sum'] = [1 if pd.isnull(x) else 0 for x in train['DOD']]
train['age'] = ((train['ClaimStartDt'] - train['DOB'])/np.timedelta64(1,'Y')).apply(np.floor)
train['Gender'][train['Gender'] == 2] = 0
train['No_days_admitted'] = (train['DischargeDt']) - (train['AdmissionDt'])
train['No_days_admitted'] = train['No_days_admitted']/np.timedelta64(1,'D')
train['No_days_admitted'][train['No_days_admitted'].isnull()==True]=1
train['No_days_ClaimProcessing'] = (train['ClaimEndDt']-train['ClaimStartDt'])
train['No_days_ClaimProcessing'] = train['No_days_ClaimProcessing']/np.timedelta64(1,'D')
train['RenalDiseaseIndicator'].unique()
train['RenalDiseaseIndicator'][train['RenalDiseaseIndicator'] == 'Y'] = 1

In [14]:
def series_mode(x):
    value = x.mode()
    if len(value)>0:
        return x.mode()[0]
    else:
        raise Exception(value)

In [15]:
def series_median(x):
    return x.median()

In [16]:
def series_sum(x):
    return x.sum()

In [ ]:
# Grouping by Providers
ee1 = train.groupby('Provider').agg({
    "age":[("age_mode",series_mode),("age_median", series_median)],
    "deceased_sum":[("deceased_sum",series_sum)],
    "Gender":[("Gender_mode", series_mode)],
    "Race":[("Race_mode", series_mode)],
    "State":[("State_mode", series_mode)],
    "County":[("County_mode", series_mode)],
    "No_days_admitted":[("No_days_admitted_median", series_median), ("No_days_admitted_mode", series_mode)],
    "No_days_ClaimProcessing":[("No_days_ClaimProcessing_median", series_median), ("No_days_ClaimProcessing_mode", series_mode)],
    "AttendingPhysician":[("AttendingPhysician_mode", series_mode)],
    "RenalDiseaseIndicator":[("RenalDiseaseIndicator_mode",series_mode)],
    "ChronicCond_Alzheimer":[("ChronicCond_Alzheimer_median", series_median), ("ChronicCond_Alzheimer_mode", series_mode)],
    "ChronicCond_Cancer":[("ChronicCond_Cancer_median", series_median), ("ChronicCond_Cancer_mode", series_mode)],
    "ChronicCond_Depression":[("ChronicCond_Depression_median", series_median), ("ChronicCond_Depression_mode", series_mode)],
    "ChronicCond_Diabetes":[("ChronicCond_Diabetes_median", series_median), ("ChronicCond_Diabetes_mode", series_mode)],
    "ChronicCond_Heartfailure":[("ChronicCond_Heartfailure_median", series_median), ("ChronicCond_Heartfailure_mode", series_mode)],
    "ChronicCond_IschemicHeart":[("ChronicCond_IschemicHeart_median", series_median), ("ChronicCond_IschemicHeart_mode", series_mode)],
    "ChronicCond_KidneyDisease":[("ChronicCond_KidneyDisease_median", series_median), ("ChronicCond_KidneyDisease_mode", series_mode)],
    "ChronicCond_ObstrPulmonary":[("ChronicCond_ObstrPulmonary_median", series_median), ("ChronicCond_ObstrPulmonary_mode", series_mode)],
    "ChronicCond_Osteoporasis":[("ChronicCond_Osteoporasis_median", series_median), ("ChronicCond_Osteoporasis_mode", series_mode)],
    "ChronicCond_rheumatoidarthritis":[("ChronicCond_rheumatoidarthritis_median", series_median), ("ChronicCond_rheumatoidarthritis_mode", series_mode)],
    "ChronicCond_stroke":[("ChronicCond_stroke_median", series_median), ("ChronicCond_stroke_mode", series_mode)],
    "NoOfMonths_PartACov":[("NoOfMonths_PartACov_median", series_median), ("NoOfMonths_PartACov_mode", series_mode)],
    "NoOfMonths_PartBCov":[("NoOfMonths_PartBCov_median", series_median), ("NoOfMonths_PartBCov_mode", series_mode)],
    "InscClaimAmtReimbursed":[("InscClaimAmtReimbursed_sum", series_sum), ("InscClaimAmtReimbursed_median", series_median)],
    "OPAnnualDeductibleAmt":[("OPAnnualDeductibleAmt_sum", series_sum), ("OPAnnualDeductibleAmt_median", series_median)],
    "OPAnnualReimbursementAmt":[("OPAnnualReimbursementAmt_sum", series_sum), ("OPAnnualReimbursementAmt_median", series_median)],
    "IPAnnualDeductibleAmt":[("IPAnnualDeductibleAmt_sum", series_sum), ("IPAnnualDeductibleAmt_median", series_median)],
    "IPAnnualReimbursementAmt":[("IPAnnualReimbursementAmt_sum", series_sum), ("IPAnnualReimbursementAmt_median", series_median)],
    "DeductibleAmtPaid":[("DeductibleAmtPaid_sum", series_sum), ("DeductibleAmtPaid_median", series_median)]
})


ee1.columns = ee1.columns.droplevel()

#For OperatingPhysician
OperatingPhysician_mode=pd.DataFrame(train.groupby(['Provider'])["OperatingPhysician"].apply(lambda x: list(x.mode())))
OperatingPhysician_mode['Provider']=OperatingPhysician_mode.index
OperatingPhysician_mode=OperatingPhysician_mode.reset_index(drop=True)
OperatingPhysician_mode.columns=["OperatingPhysician_mode", "Provider"]
OperatingPhysician_mode["Provider"]=OperatingPhysician_mode["Provider"].astype(object)
ee1=ee1.merge(OperatingPhysician_mode, on='Provider')


#For OtherPhysician
OtherPhysician_mode=pd.DataFrame(train.groupby(['Provider'])["OtherPhysician"].apply(lambda x: list(x.mode())))
OtherPhysician_mode['Provider']=OtherPhysician_mode.index
OtherPhysician_mode=OtherPhysician_mode.reset_index(drop=True)
OtherPhysician_mode.columns=["OtherPhysician_mode", "Provider"]
OtherPhysician_mode["Provider"]=OtherPhysician_mode["Provider"].astype(object)
ee1=ee1.merge(OtherPhysician_mode, on='Provider')


#no_of_service_provided
no_of_service_provided = pd.DataFrame(train['Provider'].value_counts())
no_of_service_provided.columns = ['no_of_service_provided']
no_of_service_provided['Provider'] = no_of_service_provided.index
ee1=ee1.merge(no_of_service_provided, on='Provider')





In [18]:
ee1.head().transpose()

,0,1,2,3,4
Provider,PRV51001,PRV51003,PRV51004,PRV51005,PRV51007
age_mode,69,69,72,67,71
age_median,79,71,72,70,69.5
deceased_sum,25,131,148,1161,71
Gender_mode,0,0,0,0,0
Race_mode,1,1,1,1,1
State_mode,1,1,1,1,1
County_mode,440,340,150,620,380
No_days_admitted_median,1,1,1,1,1
No_days_admitted_mode,1,1,1,1,1


In [19]:
train_df = pd.DataFrame({'Provider': list(train['Provider'].unique())})

In [20]:
#ee1.columns = ee1.columns.droplevel()
train_df = train_df.merge(ee1, on = 'Provider')

In [21]:
train_df = pd.merge(train_df, train_classification, left_on = 'Provider', right_on = 'Provider', how = 'inner')

In [22]:
train_df.head().transpose()

,0,1,2,3,4
Provider,PRV55912,PRV55907,PRV56046,PRV52405,PRV56614
age_mode,67,81,69,71,72
age_median,75,75,70,71,72
deceased_sum,105,242,20,88,26
Gender_mode,0,0,0,0,0
Race_mode,1,1,1,1,1
State_mode,39,39,39,1,45
County_mode,510,770,230,340,780
No_days_admitted_median,2,1,3,1,1.5
No_days_admitted_mode,1,1,1,1,1


In [23]:
train_df.to_csv('C:/Users/Deepak/Desktop/SB Capstone - 1/data/Processed_Data/Train.csv')